In [1]:
import sys
sys.path.insert(0, '../../dataset/Dataset Processing/')

In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import heapq
from data_processing import *  

In [3]:
def createGraph(depotNodes ,requiredEdges, numNodes, show=True):
    G = nx.Graph()
    edges = []
    pos = {}
    reqPos = {}
    s = [1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 6, 7]
    t = [2, 3, 4, 6, 4, 5, 5, 7, 6, 8, 7, 8]
    weights = [2.3, 2, 3, 1.5, 3.2, 2.2, 3.8, 2.6, 2.2, 2.8, 1.8, 0.8]
    xData = [-2, -0.5, -1,   0, 1,  1.5, 2,   2.5];
    yData = [ 0, -2,    2.5, 0, 3, -2,   0.3, 1.5];
    for i in range(len(s)):
        edges.append((s[i], t[i], weights[i]))
    
    for i in range(1, numNodes+1):
        G.add_node(i)
        pos[i] =(xData[i-1], yData[i-1])
    
    node_color = ['y']*int(G.number_of_nodes())
    depot_node_color = node_color
    for i in range(1, len(node_color)+1):
        if i in depotNodes:
            depot_node_color[i-1] = 'g'
            
    G.add_weighted_edges_from(edges)
    labels = nx.get_edge_attributes(G,'weight')
    nx.draw_networkx(G,pos, node_color = node_color)
    nx.draw_networkx(G,pos, node_color = depot_node_color)
    nx.draw_networkx_edges(G, pos, edgelist=requiredEdges, width=3, alpha=0.5,
                                        edge_color="r")
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    if show:
        plt.figure(1)
        plt.show()
    return G,pos, depot_node_color

In [4]:
# Allocating task based on distance between base station and desired edge and UAV availability
def taskAllocation(G, depotNodes, requiredNodes, desiredEdgeTravesed, numrequiredEdges, uavsInDepotNodes):
    uavsInDepotNodesCopy = uavsInDepotNodes.copy()
    depotNodesCost = np.zeros((len(depotNodes), numrequiredEdges))
    depotPath = []
    bestPathTillDesiredEdge = []
    bestCostTillDesiredEdge = []
    if not all(uav == 0 for uav in uavsInDepotNodesCopy):
        for j in range(numrequiredEdges):
            if not desiredEdgeTravesed[j]:
                for i in range(len(depotNodes)):
                    if uavsInDepotNodesCopy[i] > 0:
                            c1 = nx.dijkstra_path_length(G, source=depotNodes[i], target=requiredNodes[j][0])
                            c2 = nx.dijkstra_path_length(G, source=depotNodes[i], target=requiredNodes[j][1])
                            c2 = np.inf
                            l = []
                            if c1 <= c2:
                                l = nx.dijkstra_path(G, source=depotNodes[i], target=requiredNodes[j][0])
                                l.append(requiredNodes[j][1])
                            else:
                                l = nx.dijkstra_path(G, source=depotNodes[i], target=requiredNodes[j][1])
                                l.append(requiredNodes[j][0])
                            depotNodesCost[i,j] = min(c1,c2)
                            depotNodesCost[i,j] += G.get_edge_data(requiredNodes[j][0], requiredNodes[j][1])['weight']
                            depotPath.append(l)
                    else:
                        depotNodesCost[i,j] = np.inf
                        depotPath.append([])
            else:
                depotNodesCost[:,j] = np.inf
                for i in range(len(depotNodes)):
                    depotPath.append([])

            if uavsInDepotNodesCopy[np.argmin(depotNodesCost[:,j])] > 0 and depotNodesCost[np.argmin(depotNodesCost[:,j]), j] != np.inf:
                uavsInDepotNodesCopy[np.argmin(depotNodesCost[:,j])] -= 1
            else:
                depotNodesCost[np.argmin(depotNodesCost[:,j]),j] = np.inf
#         if any(path for path in depotPath):
        depotPath = np.transpose(np.array(depotPath, dtype=object).reshape((numrequiredEdges, len(depotNodes))))
#         else:
#             depotPath = np.zeros((numrequiredEdges, len(depotNodes)))
        taskAllocatedtoBaseStations = []
        
        
        print("Task Allocation Algorithm Output: ")
        for i in range(numrequiredEdges):
            if not desiredEdgeTravesed[i]:
                if depotNodesCost[np.argmin(depotNodesCost[:,i]), i] != np.inf:
                    taskAllocatedtoBaseStations.append(np.argmin(depotNodesCost[:,i]))
                    bestCostTillDesiredEdge.append(depotNodesCost[taskAllocatedtoBaseStations[i],i])
                    bestPathTillDesiredEdge.append(depotPath[taskAllocatedtoBaseStations[i],i])
                    print('Allocating arc ' + str(requiredNodes[i][0]) + ' - ' + str(requiredNodes[i][1]) + ' to base station - node ' + str(depotNodes[taskAllocatedtoBaseStations[i]]))
                else:
                    taskAllocatedtoBaseStations.append(None)
                    bestCostTillDesiredEdge.append(None)
                    bestPathTillDesiredEdge.append(None)
                    print('Could not allocate edge ' + str(requiredNodes[i][0]) + ' - ' + str(requiredNodes[i][1]) + ' to any base stations')
            else:
                taskAllocatedtoBaseStations.append('Task Completed')
                bestCostTillDesiredEdge.append('Task Completed')
                bestPathTillDesiredEdge.append('Task Completed')
        
    else:
        print('No UAVs available. Task allocation not possible')
        
    return bestPathTillDesiredEdge, bestCostTillDesiredEdge

In [5]:
def augment(G, numrequiredEdges, depotNodes, uavsInDepotNodes,
                          desiredEdgeTravesed, uavs, bestPathTillDesiredEdge,
                          bestCostTillDesiredEdge, vehicleCapacity, rechargeTime, numRecharge):
    bestRoute = []
    bestRouteCost = []
    minCost = np.inf
    for j in range(numrequiredEdges):
        if not desiredEdgeTravesed[j] and bestPathTillDesiredEdge[j] != None:
            minCost = np.inf
            l = []
            for i in range(len(depotNodes)):
                c1 = nx.dijkstra_path_length(G, source=bestPathTillDesiredEdge[j][-1], target=depotNodes[i])
                if c1 <= minCost:
                    l = nx.dijkstra_path(G, source=bestPathTillDesiredEdge[j][-1], target=depotNodes[i])[1:]
                    minCost = c1
            bestRoute.append(bestPathTillDesiredEdge[j] + l)
            bestRouteCost.append(bestCostTillDesiredEdge[j] + minCost)
            if bestRouteCost[j] > vehicleCapacity:
                bestRoute[j] = None
                bestRouteCost[j] = None
            else:
                desiredEdgeTravesed[j] = True
                if uavsInDepotNodes[depotNodes.index(bestRoute[j][0])] != 0: 
                    uavsInDepotNodes[depotNodes.index(bestRoute[j][0])] -= 1
                
                if heapq.heappop(uavs[depotNodes.index(bestRoute[j][0])]) == rechargeTime:
                    numRecharge += 1
                
                uavsInDepotNodes[depotNodes.index(bestRoute[j][-1])] += 1
                heapq.heappush(uavs[depotNodes.index(bestRoute[j][-1])], rechargeTime)
        else:
            bestRoute.append(None)
            bestRouteCost.append(None)
    print("Augment Step Output: ")
    return bestRoute, bestRouteCost, uavs, desiredEdgeTravesed, uavsInDepotNodes, numRecharge

In [16]:
def merge(G, numrequiredEdges, depotNodes, requiredNodes, uavsInDepotNodes,
                          desiredEdgeTravesed, uavs, bestRoute,
                          bestRouteCost, vehicleCapacity, rechargeTime, numRecharge):
    bestRouteCost1 = []
    bestRoutePath1 = []
    reqNodes = []
    for i,cost in enumerate(bestRouteCost):
        if cost:
            bestRouteCost1.append(cost)
            bestRoutePath1.append(bestRoute[i])
            reqNodes.append(requiredNodes[i])
            
    bestRoute = bestRoutePath1
    bestRouteCost = bestRouteCost1
    requiredNodes = reqNodes
    reverseSortedCost = sorted(bestRouteCost, reverse=True)
    reverseSortedPaths = []
    mergedOrder = []
    reverseSortPaths = {}
    sortedOrder = []
    
    for i in range(len(reverseSortedCost)):
        sortedOrder.append(reverseSortedCost.index(bestRouteCost[i]))
    
    for i in range(len(bestRouteCost)):
        if bestRouteCost[i] >= 0:
            sortedOrder.append(bestRouteCost.index(reverseSortedCost[i]))
            reverseSortedPaths.append(bestRoute[sortedOrder[i]])

    mergedPath = [0]*math.factorial(len(requiredNodes))
    mergedCost = [0]*math.factorial(len(requiredNodes))
    p = 0
    for i in range(len(requiredNodes)-1,-1,-1):
        for j in range(i):
            for k in range(len(reverseSortedPaths[j])):
                if reverseSortedPaths[j][k] == requiredNodes[sortedOrder[j]][0] or reverseSortedPaths[j][k] == requiredNodes[sortedOrder[j]][1]:
                    mergedPath[p] = reverseSortedPaths[j][:k+2]
                    startIndex = reverseSortedPaths[j][k+1]
                    for y in range(len(mergedPath[p])-1):
                        mergedCost[p] += G.get_edge_data(mergedPath[p][y], mergedPath[p][y+1])['weight']
                    break
            print(requiredNodes)
            print(sortedOrder[i])
            c1 = nx.dijkstra_path_length(G, source=startIndex, target=requiredNodes[sortedOrder[i]][0])
            c2 = nx.dijkstra_path_length(G, source=startIndex, target=requiredNodes[sortedOrder[i]][1])
            l = []
            if c1 <= c2:
                if c1 != 0:
                    mergedPath[p] += nx.dijkstra_path(G, source=startIndex, target=requiredNodes[sortedOrder[i]][0])[1:]
                    mergedPath[p].append(requiredNodes[sortedOrder[i]][1])
                    mergedCost[p] += c1
                else:
                    mergedPath[p] += nx.dijkstra_path(G, source=startIndex, target=requiredNodes[sortedOrder[i]][1])[1:]
            else:
                if c2 != 0:
                    mergedPath[p] += nx.dijkstra_path(G, source=startIndex, target=requiredNodes[sortedOrder[i]][1])[1:]
                    mergedPath[p].append(requiredNodes[sortedOrder[i]][0])
                    mergedCost[p] += c2
                else:
                    mergedPath[p] += nx.dijkstra_path(G, source=startIndex, target=requiredNodes[sortedOrder[i]][0])[1:]

            mergedCost[p] += G.get_edge_data(requiredNodes[sortedOrder[i]][0], requiredNodes[sortedOrder[i]][1])['weight']
            minCost = np.inf
            l = []
            for q in range(len(depotNodes)):
                c1 = nx.dijkstra_path_length(G, source=mergedPath[p][-1], target=depotNodes[q])
                if c1 <= minCost:
                    l = nx.dijkstra_path(G, source=mergedPath[p][-1], target=depotNodes[q])[1:]
                    minCost = c1
            mergedPath[p] += l
            mergedCost[p] += minCost
            mergedOrder.append([i, j])
            p += 1
    
    mergedPath = [i for i in mergedPath if i!= 0]
    mergedCost = [i for i in mergedCost if i!= 0]
    print("Merge Step Output: ")
    
    allPath = []
    allPathCost = []
    finalPath = []
    finalPathCost = []
    finalCost = [0]*len(mergedCost)

    for j in range(len(mergedCost)):
        allPath = []
        cost = []
        if mergedCost[j] <= vehicleCapacity:
            allPath.append(mergedPath[j])
            cost.append(mergedCost[j])
            for i in range(len(requiredNodes)):
                if i not in mergedOrder[j]:
                    allPath.append(bestRoute[sortedOrder[i]])
                    cost.append(bestRouteCost[sortedOrder[i]])
                    finalCost[j] += mergedCost[j] + bestRouteCost[sortedOrder[i]]
            finalPath.append(allPath)
            finalPathCost.append(cost)
    if not finalPath:
        return bestRoute, bestRouteCost, uavs, desiredEdgeTravesed, uavsInDepotNodes, numRecharge
    else:
        print(finalPath)
        print(finalPathCost)
        print(finalCost)
        finalCost = [i for i in finalCost if i!= 0]
        if not finalCost:
            finalCost.append(0)
        return finalPath[finalCost.index(min(finalCost))], finalPathCost[finalCost.index(min(finalCost))], uavs, desiredEdgeTravesed, uavsInDepotNodes, numRecharge

In [17]:
def visualizePath(depotNodes, requiredNodes, numNodes, path, pathType="solution"):
    plt.figure(1)
    for j in range(len(path)):
        if path[j] != None:
#             plt.figure(j+1)
            G, pos, depot_node_color = createGraph(depotNodes, requiredNodes , numNodes, show=False)
            G1 = nx.DiGraph()
            pos1 = {}
            node_color = []
            edges = []
            for i in range(len(path[j])-1):
                edges.append((path[j][i], path[j][i+1], G.get_edge_data(path[j][i], path[j][i+1])['weight']))
                pos1[path[j][i]] = pos[path[j][i]]
                if i == len(path[j])-2:
                    pos1[path[j][i+1]] = pos[path[j][i+1]]

            for key in pos1.keys():
                node_color.append(depot_node_color[key-1])

            G1.add_weighted_edges_from(edges)
            nx.draw_networkx(G1,pos1, arrows=True, node_color = node_color, edge_color='b', arrowsize=12, width=1, arrowstyle='simple')
            if pathType == "solution":
                plt.legend(["Solution Path"], loc ="upper left")
            else:
                plt.legend(["Path"], loc ="upper left")
            plt.show()

In [18]:
# def main():
#     vehicleCapacity = 14
#     numNodes = 8
#     requiredNodes = [[2, 4], [6, 7], [3, 4]];
#     uavsInDepotNodes = [0, 2];
#     totalUavs = sum(uavsInDepotNodes);
#     numrequiredEdges = len(requiredNodes);
#     depotNodes = [1, 5];
#     G,pos, depot_node_color = createGraph(depotNodes, requiredNodes, numNodes, show=True)
#     bestPathTillDesiredEdge, bestCostTillDesiredEdge = taskAllocation(G, depotNodes, requiredNodes, numrequiredEdges)
#     visualizePath(depotNodes, requiredNodes, numNodes, bestPathTillDesiredEdge, pathType="normal")
#     bestRoute, bestRouteCost = augment(G, numrequiredEdges, depotNodes, bestPathTillDesiredEdge, bestCostTillDesiredEdge, vehicleCapacity)
#     print(bestRouteCost)
#     visualizePath(depotNodes, requiredNodes, numNodes, bestRoute)
#     mergedPath, mergedCost, mergedOrder, sortedOrder = merge(G, bestRoute, depotNodes, bestRouteCost, requiredNodes, numrequiredEdges)
#     visualizePath(depotNodes, requiredNodes, numNodes, mergedPath)
#     finalPath, finalCost = finalPathAndCost(mergedPath, mergedCost, mergedOrder, sortedOrder, bestRoute, bestRouteCost, numrequiredEdges)
#     print(finalCost)
#     visualizePath(depotNodes, requiredNodes, numNodes, finalPath)

# if __name__ == "__main__":
#     # execute only if run as a script
#     main()

In [19]:
def main():
    # Initializing Parameters
#     vehicleCapacity = 14
#     numNodes = 8
#     requiredNodes = [[2, 4], [6, 7], [3, 4]]
#     depotNodes = [1, 5]
#     uavsInDepotNodes = [1, 1]
#     rechargeTime = 20
      
    position = list(np.load('../../dataset/graph info/pos.npy', allow_pickle=True))
    nodeColor = list(np.load('../../dataset/graph info/node_color.npy', allow_pickle=True))
    depotNodeColor = list(np.load('../../dataset/graph info/depot_node_color.npy', allow_pickle=True))
    Edges = list(np.load('../../dataset/graph info/edges.npy', allow_pickle=True))
    vehCap = list(np.load('../../dataset/graph info/vehicleCapacity.npy', allow_pickle=True))
    nuNod = list(np.load('../../dataset/graph info/numNodes.npy', allow_pickle=True))
    reqNod = list(np.load('../../dataset/graph info/requiredNodes.npy', allow_pickle=True))
    deNo = list(np.load('../../dataset/graph info/depotNodes.npy', allow_pickle=True))
    
    info = readAndStoreInstanceInfo('../../dataset/CARP_datasets/DeArmon_gdb-IF')
    instanceData = {'Instance Name' : [],
                    'Number of Nodes' : [],
                    'Number of Edges' : [],
                    'Number of Required Edges' : [],
                    'Capacity' : [],
                    'Number of Depot Nodes' : []}

    algoOutputData = {'Execution Time' : [],
                      'Maximum Cost' : [],
                      'Total Cost' : [],
                      'Number of Recharges' : [],
                      'Number of Desired Edges Traversed' : [],
                      'Success Rate' : []}
    
    folderPath = '../../dataset/graph files'
    for i, file in enumerate(os.listdir(folderPath)):
        if file.endswith(".pkl"):
            index = int(file.split('.')[0])
            file_path = f"{folderPath}/{file}"
            
            G = nx.read_gpickle(file_path)
            pos = position[index]
            node_color = nodeColor[index]
            depot_node_color = depotNodeColor[index]
            edges = Edges[index]
            depotNodes = deNo[index]
            requiredNodes = reqNod[index]
            vehicleCapacity = vehCap[index]
            numNodes = nuNod[index]
            uavsInDepotNodes = [len(requiredNodes)//(2*len(depotNodes))]*len(depotNodes)
            rechargeTime = 2*vehicleCapacity
            instanceData['Instance Name'].append(info['Instance Name'][index])
            instanceData['Number of Nodes'].append(numNodes)
            instanceData['Number of Edges'].append(len(edges))
            instanceData['Number of Required Edges'].append(len(requiredNodes))
            instanceData['Capacity'].append(vehicleCapacity)
            instanceData['Number of Depot Nodes'].append(len(depotNodes))
            
#     G,pos, node_color, depot_node_color, edges, depotNodes, requiredNodes, vehicleCapacity, numNodes = createGraph()
#     uavsInDepotNodes = [len(requiredNodes)//(2*len(depotNodes))]*len(depotNodes)
#     rechargeTime = 2*vehicleCapacity
    
            # Declaring useful paramerters
            numrequiredEdges = len(requiredNodes)
            desiredEdgeTravesed = [False]*numrequiredEdges
            numRecharge = 0
            iterations = 0
            uavs = []
            allRoutes = []
            allRoutesCosts = []
            for element in uavsInDepotNodes:
                l = [0]*element
                heapq.heapify(l)
                uavs.append(l)
            penality = 0
            print(sum(uavsInDepotNodes))
            start = time.time()
        #     G,pos, depot_node_color = createGraph(depotNodes, requiredNodes, numNodes, show=False)

            while not all(edges == True for edges in desiredEdgeTravesed):
                if iterations <= sum(uavsInDepotNodes)*numrequiredEdges:
                    bestPathTillDesiredEdge, bestCostTillDesiredEdge = taskAllocation(G, depotNodes, requiredNodes, desiredEdgeTravesed, numrequiredEdges, uavsInDepotNodes)
            #         visualizePath(depotNodes, requiredNodes, numNodes, bestPathTillDesiredEdge, pathType="normal")
#                     bestRoute, bestRouteCost, uavs, desiredEdgeTravesed, uavsInDepotNodes, numRecharge = pathScanningAlgorithm(G, numrequiredEdges,
#                                                                                                                                  depotNodes, uavsInDepotNodes,
#                                                                                                                                  desiredEdgeTravesed, uavs,
#                                                                                                                                  bestPathTillDesiredEdge, bestCostTillDesiredEdge,
#                                                                                                                                  vehicleCapacity, rechargeTime, numRecharge)
                
                    bestRoute, bestRouteCost, uavs, desiredEdgeTravesed, uavsInDepotNodes, numRecharge = augment(G, numrequiredEdges, depotNodes, uavsInDepotNodes,
                                                                                                              desiredEdgeTravesed, uavs, bestPathTillDesiredEdge,
                                                                                                              bestCostTillDesiredEdge, vehicleCapacity, rechargeTime, numRecharge)
            
                    bestRoute, bestRouteCost, uavs, desiredEdgeTravesed, uavsInDepotNodes, numRecharge = merge(G, numrequiredEdges, depotNodes, requiredNodes, uavsInDepotNodes,
                                                                                                              desiredEdgeTravesed, uavs, bestRoute,
                                                                                                              bestRouteCost, vehicleCapacity, rechargeTime, numRecharge)
            
            
            #         visualizePath(depotNodes, requiredNodes, numNodes, bestRoute)
                    for i in range(len(bestRoute)):
                        if bestRoute[i] != None:
                            allRoutes.append(bestRoute[i])
                            allRoutesCosts.append(bestRouteCost[i])
                else:
                    break
                iterations += 1

            end = time.time()
        #     visualizePath(depotNodes, requiredNodes, numNodes, allRoutes)
            if i == 10:
                print(instanceData['Instance Name'][-1])
#                 visualizePath(G,pos, node_color, depot_node_color, edges, depotNodes, requiredNodes, numNodes, allRoutes)
            print(str(desiredEdgeTravesed.count(True)) + ' out of '+ str(len(desiredEdgeTravesed)) + ' is Traversed.')
            print('Success rate is ' + str((desiredEdgeTravesed.count(True)/len(desiredEdgeTravesed))*100) + '%.')
            print('Maximum of all path cost = ' + str(max(allRoutesCosts)))
            print('Number of times drones had to recharge = ' + str(numRecharge))
            print('Total cost of all paths + recharge cost = ' + str(sum(allRoutesCosts) + rechargeTime*numRecharge))
            print("Execution took "+ str(end-start) + " seconds.")
            algoOutputData['Execution Time'].append(round(end-start, 3))
            algoOutputData['Maximum Cost'].append(max(allRoutesCosts))
            algoOutputData['Total Cost'].append(sum(allRoutesCosts) + rechargeTime*numRecharge)
            algoOutputData['Number of Recharges'].append(numRecharge)
            algoOutputData['Number of Desired Edges Traversed'].append(desiredEdgeTravesed.count(True))
            algoOutputData['Success Rate'].append((desiredEdgeTravesed.count(True)/len(desiredEdgeTravesed))*100)
            print()
            print()
#             df = pd.DataFrame(instanceData)
#             df.to_csv('../../dataset/dataset_info.csv')
            
            df = pd.DataFrame(algoOutputData)
            df.to_csv('../../dataset/algo_output_info.csv')
            
if __name__ == "__main__":
    # execute only if run as a script
    main()

Index(['Instance Name', 'Number of Nodes', 'Number of Edges', 'Required Edges',
       'Capacity', 'Number of Depot Nodes', 'Depot Nodes'],
      dtype='object')
3
Task Allocation Algorithm Output: 
Allocating arc 1 - 2 to base station - node 1
Allocating arc 1 - 10 to base station - node 9
Allocating arc 2 - 9 to base station - node 11
Could not allocate edge 3 - 4 to any base stations
Could not allocate edge 5 - 11 to any base stations
Could not allocate edge 7 - 8 to any base stations
Could not allocate edge 7 - 12 to any base stations
Could not allocate edge 8 - 11 to any base stations
Augment Step Output: 
[[1, 2], [2, 9]]
0
Merge Step Output: 
[[[11, 9, 2, 1]]]
[[29]]
[0]
Task Allocation Algorithm Output: 
Allocating arc 1 - 10 to base station - node 9
Allocating arc 3 - 4 to base station - node 9
Allocating arc 5 - 11 to base station - node 9
Could not allocate edge 7 - 8 to any base stations
Could not allocate edge 7 - 12 to any base stations
Could not allocate edge 8 - 11 to a

Augment Step Output: 
[[1, 2], [1, 3], [1, 5], [1, 7]]
0
[[1, 2], [1, 3], [1, 5], [1, 7]]
0
[[1, 2], [1, 3], [1, 5], [1, 7]]
0
[[1, 2], [1, 3], [1, 5], [1, 7]]
0
[[1, 2], [1, 3], [1, 5], [1, 7]]
0
[[1, 2], [1, 3], [1, 5], [1, 7]]
2
Merge Step Output: 
[[[2, 1, 7, 1, 2], [2, 1, 5, 1], [1, 2]], [[2, 1, 5, 1, 2], [2, 1, 7, 1], [1, 2]], [[1, 2, 1], [2, 1, 7, 1], [2, 1, 5, 1]], [[2, 1, 7, 1, 2], [2, 1, 5, 1], [1, 2]], [[2, 1, 5, 1, 2], [2, 1, 7, 1], [1, 2]], [[2, 1, 7, 1, 5, 1], [1, 2], [1, 2]]]
[[4, 3.0, 1.0], [4, 3.0, 1.0], [2, 3.0, 3.0], [4, 3.0, 1.0], [4, 3.0, 1.0], [5, 1.0, 1.0]]
[12.0, 12.0, 10.0, 12.0, 12.0, 12.0]
Task Allocation Algorithm Output: 
Allocating arc 2 - 3 to base station - node 2
Allocating arc 2 - 7 to base station - node 1
Allocating arc 5 - 6 to base station - node 1
Allocating arc 5 - 7 to base station - node 1
Augment Step Output: 
[[2, 3], [2, 7], [5, 6], [5, 7]]
0
[[2, 3], [2, 7], [5, 6], [5, 7]]
0
[[2, 3], [2, 7], [5, 6], [5, 7]]
0
[[2, 3], [2, 7], [5, 6], [5, 7

[[1, 4], [1, 9], [1, 11], [2, 4], [2, 5], [3, 7]]
2
[[1, 4], [1, 9], [1, 11], [2, 4], [2, 5], [3, 7]]
0
[[1, 4], [1, 9], [1, 11], [2, 4], [2, 5], [3, 7]]
0
[[1, 4], [1, 9], [1, 11], [2, 4], [2, 5], [3, 7]]
0
[[1, 4], [1, 9], [1, 11], [2, 4], [2, 5], [3, 7]]
0
[[1, 4], [1, 9], [1, 11], [2, 4], [2, 5], [3, 7]]
4
[[1, 4], [1, 9], [1, 11], [2, 4], [2, 5], [3, 7]]
4
[[1, 4], [1, 9], [1, 11], [2, 4], [2, 5], [3, 7]]
4
[[1, 4], [1, 9], [1, 11], [2, 4], [2, 5], [3, 7]]
1
[[1, 4], [1, 9], [1, 11], [2, 4], [2, 5], [3, 7]]
1
[[1, 4], [1, 9], [1, 11], [2, 4], [2, 5], [3, 7]]
3
Merge Step Output: 
[[[4, 2, 5, 1, 11, 7], [7, 2, 4], [1, 9, 4], [4, 2, 5, 1], [1, 4]], [[7, 2, 4, 1, 11, 7], [4, 2, 5, 1], [1, 9, 4], [4, 2, 5, 1], [1, 4]], [[1, 9, 1, 11, 7], [4, 2, 5, 1], [7, 2, 4], [4, 2, 5, 1], [1, 4]], [[4, 2, 5, 1, 11, 7], [4, 2, 5, 1], [7, 2, 4], [1, 9, 4], [1, 4]], [[1, 4, 1, 11, 7], [4, 2, 5, 1], [7, 2, 4], [1, 9, 4], [4, 2, 5, 1]], [[4, 2, 5, 1, 4], [7, 2, 4], [1, 9, 4], [4, 2, 5, 1], [7, 1, 11, 7

Task Allocation Algorithm Output: 
Allocating arc 1 - 4 to base station - node 7
Allocating arc 7 - 8 to base station - node 3
Allocating arc 8 - 11 to base station - node 3
Could not allocate edge 9 - 10 to any base stations
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 1 - 4 to base station - node 7
Allocating arc 7 - 8 to base station - node 3
Allocating arc 8 - 11 to base station - node 3
Could not allocate edge 9 - 10 to any base stations
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 1 - 4 to base station - node 7
Allocating arc 7 - 8 to base station - node 3
Allocating arc 8 - 11 to base station - node 3
Could not allocate edge 9 - 10 to any base stations
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 1 - 4 to base station - node 7
Allocating arc 7 - 8 to base station - node 3
Allocating arc 8 - 11 to base station - node 3
Could not allocate edg

Could not allocate edge 8 - 10 to any base stations
Could not allocate edge 8 - 11 to any base stations
Could not allocate edge 9 - 10 to any base stations
Could not allocate edge 9 - 11 to any base stations
Could not allocate edge 10 - 11 to any base stations
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 1 - 7 to base station - node 4
Allocating arc 3 - 5 to base station - node 4
Allocating arc 8 - 10 to base station - node 11
Could not allocate edge 8 - 11 to any base stations
Could not allocate edge 9 - 10 to any base stations
Could not allocate edge 9 - 11 to any base stations
Could not allocate edge 10 - 11 to any base stations
Augment Step Output: 
[[3, 5], [8, 10]]
1
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 1 - 7 to base station - node 12
Allocating arc 8 - 11 to base station - node 11
Allocating arc 9 - 10 to base station - node 4
Could not allocate edge 9 - 11 to any base stations
Could not allocate edg

Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 21 - 23 to base station - node 13
Allocating arc 23 - 26 to base station - node 1
Allocating arc 26 - 27 to base station - node 13
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 21 - 23 to base station - node 13
Allocating arc 23 - 26 to base station - node 1
Allocating arc 26 - 27 to base station - node 13
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 21 - 23 to base station - node 13
Allocating arc 23 - 26 to base station - node 1
Allocating arc 26 - 27 to base station - node 13
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 21 - 23 to base station - node 13
Allocating arc 23 - 26 to base station - node 1
Allocating arc 26 - 27 to base station - node 13
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 21 - 23 to base 

Merge Step Output: 
[[[9, 5, 4, 12, 2, 1, 27, 8], [9, 5, 4, 12, 2, 1], [24, 1, 19, 4, 12, 19, 1], [1, 3, 8]], [[9, 5, 4, 12, 2, 1, 27, 8], [9, 5, 4, 12, 2, 1], [24, 1, 19, 4, 12, 19, 1], [1, 3, 8]], [[9, 5, 4, 12, 2, 1, 19, 12, 4, 19, 1], [9, 5, 4, 12, 2, 1], [1, 3, 8], [17, 1, 27, 8]], [[9, 5, 4, 12, 2, 1, 19, 12, 4, 19, 1], [9, 5, 4, 12, 2, 1], [1, 3, 8], [17, 1, 27, 8]], [[9, 5, 4, 12, 2, 1, 2, 1], [24, 1, 19, 4, 12, 19, 1], [1, 3, 8], [17, 1, 27, 8]]]
[[18, 10.0, 14.0, 13.0], [18, 10.0, 14.0, 13.0], [18, 10.0, 13.0, 13.0], [18, 10.0, 13.0, 13.0], [16, 14.0, 13.0, 13.0]]
[91.0, 91.0, 0, 0, 0, 0, 0, 90.0, 90.0, 88.0]
Task Allocation Algorithm Output: 
Allocating arc 4 - 19 to base station - node 1
Allocating arc 5 - 9 to base station - node 9
Allocating arc 6 - 10 to base station - node 8
Allocating arc 7 - 11 to base station - node 8
Allocating arc 8 - 27 to base station - node 22
Allocating arc 13 - 15 to base station - node 1
Could not allocate edge 16 - 17 to any base stations
Co

Task Allocation Algorithm Output: 
Allocating arc 7 - 11 to base station - node 9
Allocating arc 21 - 25 to base station - node 22
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 7 - 11 to base station - node 9
Allocating arc 21 - 25 to base station - node 22
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 7 - 11 to base station - node 9
Allocating arc 21 - 25 to base station - node 22
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 7 - 11 to base station - node 9
Allocating arc 21 - 25 to base station - node 22
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 7 - 11 to base station - node 9
Allocating arc 21 - 25 to base station - node 22
Augment Step Output: 
Merge Step Output: 
Task Allocation Algorithm Output: 
Allocating arc 7 - 11 to base station - node 9
Allocating arc 21 - 25 to base station - node 22
Augm